In [ ]:
import pymysql
import pandas as pd
from DBdata2 import *
import requests

In [ ]:
token = 'd946e212-242e-4094-8c30-d990cec12983-843339462' # colocar aquí el token
url_endpoint = 'https://tagme.d4science.org/tagme/tag?lang=en&include_abstract=true&include_categories=true&gcube-token='
headers = {'user-agent': 'Mozilla/5.0', 'accept': 'application/json', 'content-type': 'application/json'}
dbr = 'http://dbpedia.org/resource/'
dbc = 'http://dbpedia.org/resource/Category:'

global data
data = Datos2()

In [ ]:
def getAnnotations(paper_id, text):
    url = url_endpoint + token + '&text=' + text

    resp = requests.get(url, headers=headers).json()
    resp.keys()
    annotations = resp['annotations']

    dbCategories = []
    dbResources = []
    try:
        for i in range(len(annotations)):
            ann = annotations[i]['spot']
            if annotations[i]['rho'] > 0.1 and annotations[i]['link_probability'] > 0.1:
                if ann.islower(): ann = ann.capitalize() 
                uridb = dbr + ann.replace(' ', '_')
                dbResources.append([paper_id, uridb, annotations[i]['rho'],
                                     annotations[i]['link_probability'], ann])
                # Guardar como recursos de la Dbpedia:
                if 'dbpedia_categories' in annotations[i]:
                
                    if len(annotations[i]['dbpedia_categories']):
                        for c in annotations[i]['dbpedia_categories']:
                            if c.islower(): c = c.capitalize() 
                            dbCategories.append([paper_id, dbc + c.replace(' ', '_'), annotations[i]['rho'],
                                             annotations[i]['link_probability'], c])
    except KeyError:
        print(annotations[i])
        print(annotations[i]['dbpedia_categories'], "Error")        
    if len(dbResources):
        dbResources = pd.DataFrame(dbResources, columns=['paperID', 'uri', 'rho', 'link_probability', 'label'])
    if len(dbCategories):
        dbCategories = pd.DataFrame(dbCategories, columns=['paperID', 'uri', 'rho', 'link_probability', 'label'])

    return dbResources, dbCategories

In [ ]:
db = pymysql.connect(host='localhost', user='root', password='', db='data')
cur=db.cursor()
cur.execute(u"""SELECT DISTINCT sujeto, group_concat(IF(predicado = 'https://w3id.org/scholarlydata/ontology/conference-ontology.owl#title', objeto, '')) 
titulo, group_concat(IF(predicado = 'https://w3id.org/scholarlydata/ontology/conference-ontology.owl#abstract', objeto, '')) 
resumen FROM tripletas WHERE predicado = 'https://w3id.org/scholarlydata/ontology/conference-ontology.owl#abstract' 
OR  predicado = 'https://w3id.org/scholarlydata/ontology/conference-ontology.owl#title' GROUP BY sujeto;""")

In [ ]:
filas = cur.fetchall()
for fila in filas:
    paper = fila[0]
    titulo = fila[1].strip(',')
    resumen = fila[2].strip(',')

    listAbs = resumen.split('.')
    listTitu= titulo
    for frase in listAbs:
        paper_id = paper
        text = frase
        if text.strip():
            dbResources, dbCategories = getAnnotations(paper_id, text)
            if len(dbResources)>0 and len(dbCategories)>0:   
                dbResourcesSelected = dbResources[dbResources['link_probability']>0.3]
                dbCategoriesSelected = dbCategories[dbCategories['link_probability']>0.3]

In [ ]:
tripletsR = []
for i, r in dbResourcesSelected.iterrows():
    tripletsR.append([r['paperID'], '<http://schema.org/mentions>', '<'+r['uri']+'>'])
    tripletsR.append([r['uri'], '<http://www.w3.org/2000/01/rdf-schema#label>', ' "'+ r['label']+'"'])
fileN = open("dbpediaR.nt","a")

for t in tripletsR:
    fileN.writelines('<'+ t[0]+'> '+t[1]+t[2]+' .\n')
fileN.close()

In [ ]:
tripletsC = []
for i, r in dbCategoriesSelected.iterrows():
    tripletsC.append([r['paperID'], '<http://schema.org/mentions>', ' <'+r['uri']+'>'])
    tripletsC.append([r['uri'], '<http://www.w3.org/2000/01/rdf-schema#label>', ' "'+r['label']+'"'])

fileN = open("dbpediaC.nt","a")

for t in tripletsC:
    fileN.writelines('<' + t[0]+'> ' + t[1] + t[2] + '.\n')
fileN.close()